# Failed Runs Checking

## 1. Failed jobs

#### failed files:

In [ ]:
(base) kexdai@eu-login-40:/cluster/scratch/kexdai$ cat Cheese_PRJNA997801_sequences.err
Plugin error from fondue:

  /scratch/tmp.24135457.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-oxfejuiu is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR25395177', 'SRR25395174'}

Debug info has been saved to /scratch/tmp.24135457.kexdai/qiime2-q2cli-err-x2ru7fkn.log
(base) kexdai@eu-login-40:/cluster/scratch/kexdai$ cat Cheese_PRJNA1094419_sequences.err
Plugin error from fondue:

  /scratch/tmp.24135453.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-fnlpv54e is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR28523246', 'SRR28523251'}

Debug info has been saved to /scratch/tmp.24135453.kexdai/qiime2-q2cli-err-a75vszba.log

I have removed the two sample ids SRR28523246 and SRR28523251 from PRJNA1094419, and I set the running time for this BioProject for 2 weeks (because it timed out for 6-day running)

In [ ]:
Plugin error from fondue:

  /scratch/tmp.24997728.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-c36iltio is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR25394842', 'SRR25394843'}

Debug info has been saved to /scratch/tmp.24997728.kexdai/qiime2-q2cli-err-nxfnq2x6.log

I remove the four problematic run ids, and run again for PRJNA997801
(the metadata artifact and table are also without 'SRR25395177', 'SRR25395174', 'SRR25394842', 'SRR25394843')

In [ ]:
Plugin error from fondue:

  /scratch/tmp.25119041.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-403ujr21 is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR25394840', 'SRR25394841'}

Debug info has been saved to /scratch/tmp.25119041.kexdai/qiime2-q2cli-err-qgczhyp8.log

I remove the failed run ids in the five cheese id file for this BioProject, but I highlight them in the excel file 

1:

In [ ]:
Plugin error from fondue:

  /scratch/tmp.25288823.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-fl9ojzr9 is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR25395503', 'SRR25395505', 'SRR25395499'}

Debug info has been saved to /scratch/tmp.25288823.kexdai/qiime2-q2cli-err-ud99s_5x.log

I haven't move away these three in the .tsv id file, I uploaded this file again and reget the id artifact and set the sequence fetching job running again.

2:

In [ ]:
Plugin error from fondue:

  /scratch/tmp.25312344.kexdai/q2-CasavaOneEightSingleLanePerSampleDirFmt-5n2a9byp is not a(n) CasavaOneEightSingleLanePerSampleDirFmt:

  These samples do not have matching pairs of forward and reverse reads: {'SRR25395238'}

Debug info has been saved to /scratch/tmp.25312344.kexdai/qiime2-q2cli-err-i8j2b9bu.log

removed, and rerun

### Summary for exclued IDs for sequence fetching using fondue

PRJNA997801:
'SRR25395177', 'SRR25395174', 'SRR25394842', 'SRR25394843', 'SRR25394840', 'SRR25394841', 'SRR25395503', 'SRR25395505', 'SRR25395499', 'SRR25395238'

PRJNA1094419:
'SRR28523246', 'SRR28523251'

PRJNA997801:
SRR25395320, SRR25395327
'SRR25395116', 'SRR25395129'
SRR25395379
'SRR25395496', 'SRR25395493'
'SRR25394997', 'SRR25395003'

## Solution: download these with iSeq

In [ ]:
# Try installing with Mamba (a faster alternative to Conda):
conda install -n base -c conda-forge mamba
# Then, install iseq using Mamba:
mamba install -c bioconda iseq
# Try creating a clean environment first
# Instead of installing in an existing environment, create a fresh one:
conda create -n iseq_env -c bioconda -c conda-forge iseq
conda activate iseq_env

I run iSeq for one Run Accession

In [ ]:
iseq -i SRR28523246 -q

output files: 
SRR28523246
SRR28523246_1.fastq
SRR28523246_2.fastq
SRR28523246.fastq
SRR28523246.metadata.tsv
success.log

Check for the number of reads: If the number of reads in SRR28523246.fastq is the same as the sum of the forward and reverse read files (SRR28523246_1.fastq and SRR28523246_2.fastq), then it indicates that the reads are paired and combined correctly. 

By using wc -l command to check line counts, the number of reads is significantly lower in combined file SRR28523246.fastq, so this suggest some error with the matching between forward and reverese read

fetch the sequences of these error run accessions, and generate artifact (the matching will be checked during this process):

In [ ]:
# create a txt file and transfer to scratch, then use dos2unix tool to convert file to Unix-style ending
dos2unix run_accessions.txt

In [ ]:
# run iseq under ERROR-SEQUENCES folder
iseq -i run_accessions.txt -g
# use nohhup to allow contimuous running even disconnect or termial session ends
# nohup iseq -i run_accessions.txt -g &

In [ ]:
# rename the sequence files to meet qiime analysis
for f in *_1.fastq.gz; do  
    base=$(basename "$f" _1.fastq.gz)  
    mv "${base}_1.fastq.gz" "${base}_L001_R1_001.fastq.gz"  
    mv "${base}_2.fastq.gz" "${base}_L001_R2_001.fastq.gz"  
done
# create a folder for sequences
mkdir SEQUENCES
# move the sequence files inside the folder
mv *_L001_R1_001.fastq.gz *_L001_R2_001.fastq.gz SEQUENCES/

In [ ]:
# creare manifest
echo -e "SampleID\tforward-absolute-filepath\treverse-absolute-filepath" > manifest.csv
for file in *_L001_R1_001.fastq.gz; do
    sample_id=$(echo "$file" | cut -d'_' -f1)
    echo -e "$sample_id\t/cluster/scratch/kexdai/ERROR-SEQUENCES/SEQUENCES/${sample_id}_L001_R1_001.fastq.gz\t/cluster/scratch/kexdai/ERROR-SEQUENCES/SEQUENCES/${sample_id}_L001_R2_001.fastq.gz" >> manifest.csv
done
# rename it
mv manifest.csv MANIFEST

In [ ]:
# import sequence into QIIME 2 artifact
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path MANIFEST \
  --output-path paired-end-demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

In [ ]:
# briefly examine the resulting artifact
qiime tools peek paired-end-demux.qza

In [ ]:
# to visual summary
qiime demux summarize \
  --i-data paired-end-demux.qza \
  --o-visualization paired-end-demux.qzv \
  --verbose

## 2. Failed ids

Here I am checking the failed ids in the cache, which is the fialed runs generated by fondue

PRJEB65292:

In [ ]:
ID      Error message
ERR11865387     "cursor-cache : 5,242,880 bytes
buf-size     : 1,048,576 bytes
mem-limit    : 52,428,800 bytes
threads      : 24
scratch-path : '/scratch/tmp.24794122.kexdai/tmpfj27bojx/fasterq.tmp.eu-g9-038-1.925261/'
total ram    : 402,982,465,536 bytes
output-format: FASTQ split 3
check-mode   : on
output-file  : '/scratch/tmp.24794122.kexdai/tmpfj27bojx/ERR11865387.fastq'
output-dir   : '.'
output       : '/scratch/tmp.24794122.kexdai/tmpfj27bojx/ERR11865387.fastq'
append-mode  : 'NO'
stdout-mode  : 'NO'
seq-defline  : '@$ac.$si $sn length=$rl'
qual-defline  : '+$ac.$si $sn length=$rl'
only-unaligned : 'NO'
only-aligned   : 'NO'
accession     : 'ERR11865387'
accession-path: 'ERR11865387'
est. output          : 2,025,515,896 bytes
disk-limit (OS)      : 1,188,997,259,264 bytes
disk-limit-tmp (OS)  : 1,188,997,918,720 bytes
out/tmp on same fs   : 'NO'

ERR11865387 is remote
... has a size of 289,071,774 bytes
... is cSRA without alignments
... SEQ has NAME column = YES
... SEQ has SPOT_GROUP column = YES
... uses 'SEQUENCE' as sequence-table
SEQ.first_row = 1
SEQ.row_count = 2,272,632
SEQ.spot_count = 2,272,632
SEQ.total_base_count = 676,408,412
SEQ.bio_base_count = 676,408,412
SEQ.avg_name_len = 40
SEQ.avg_spot_group_len = 0
SEQ.avg_bio_reads_per_spot = 2
SEQ.avg_tech_reads_per_spot = 0
ALIGN.first_row = 0
ALIGN.row_count = 0
ALIGN.spot_count = 0
ALIGN.total_base_count = 0
ALIGN.bio_base_count = 0

disk-limit(s) not exeeded!
2025-02-28T18:17:18 fasterq-dump.3.0.0 err: cmn_iter.c cmn_open_db().VDBManagerOpenDBReadVPath( 'ERR11865387' ) -> RC(rcDB,rcMgr,rcOpening,rcDatabase,rcIncorrect) 
2025-02-28T18:17:18 fasterq-dump.3.0.0 err: make_fastq_sra_iter.make_cmn_iter() -> RC(rcDB,rcMgr,rcOpening,rcDatabase,rcIncorrect)
2025-02-28T18:17:18 fasterq-dump.3.0.0 err: make_fastq_iter() -> RC(rcDB,rcMgr,rcOpening,rcDatabase,rcIncorrect)
spots read      : 2,177,938
reads read      : 4,355,876
reads written   : 4,355,876
fasterq-dump quit with error code 3
"

PRJNA1052643:

error code 3: SRR27593168, SRR27593194
SRR27593235     "2025-02-27T04:56:45 fasterq-dump.3.0.0 fatal: SIGNAL - 8 
fasterq-dump quit with error code 1

PRJNA977472: (not an error but informational message)

In [ ]:
ID      Error message
SRR26215786     "2025-03-01T13:52:30 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"
SRR26215931     "2025-03-01T13:52:31 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"
SRR26215953     "2025-03-01T13:52:32 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"
SRR26216087     "2025-03-01T13:52:33 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"
SRR26216090     "2025-03-01T13:52:34 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"
SRR26216109     "2025-03-01T13:52:36 prefetch.3.0.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
"


PRJNA997801-2:

SRR25394817, SRR25394818, SRR25394835, SRR25394836, SRR25394893, SRR25394896, SRR25394898, SRR25394901, SRR25394902, SRR25394923, SRR25394985, SRR25394986, SRR25394987, SRR25395123
error code 3: SRR25394819, SRR25395125